In [ ]:
ls

In [ ]:
# Install `transformers` from master
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.11.0
# tokenizers version at notebook update --- 0.8.0rc1

#!pip install transformers==2.11.0

In [ ]:
!head -5 ben.txt

In [ ]:
with open('chunk_0981.txt', 'r', encoding = 'utf-8') as f:
  data = f.read()
print(data[:2000])

In [ ]:
lines = data.split("।")
train_data = []
for l in lines:
  train_data.append(l)
print(train_data[1])
print(len(train_data))

In [ ]:
with open('train_data.txt', 'w', encoding = 'utf-8') as f:
  data = f.write("".join(train_data))

In [ ]:
TRAIN = True
if TRAIN:
    import sentencepiece as spm
    spm.SentencePieceTrainer.train(input='train_data.txt', model_prefix='spm',vocab_size=9999) # Keep one space for mask token.

In [ ]:
from transformers import XLMRobertaConfig
TRAIN = True
# if you dont give    vocab_size=10000, this parameter here this works fine because this param is calculated from the models with masks and everything extra 
config = XLMRobertaConfig(
    vocab_size=10001, 
    max_position_embeddings=514,
    num_attention_heads=8,
    num_hidden_layers=6,
    type_vocab_size=1
)

from transformers import XLMRobertaTokenizer

tokenizer = XLMRobertaTokenizer.from_pretrained("spm.model", max_len=512)
#enable_truncation(max_length=100)
print(train_data[-1])
tokenizer._tokenize(train_data[-1])

In [ ]:


if TRAIN:
    %%time
    from transformers import TextDataset

    dataset = TextDataset(
        tokenizer=tokenizer,
        file_path="train_data.txt",
        block_size=128
    )



In [ ]:
if TRAIN:
    from transformers import DataCollatorForLanguageModeling

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=True, mlm_probability=0.15
    )

if TRAIN:  
    from transformers import Trainer, TrainingArguments

    training_args = TrainingArguments(
        output_dir="xlm_train/",
        overwrite_output_dir=True,
        num_train_epochs=1,
        per_device_train_batch_size=64,
        save_steps=5000,
        save_total_limit=2
    )

    from transformers import XLMRobertaForMaskedLM

    model = XLMRobertaForMaskedLM(config=config)

    trainer = Trainer(

        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=dataset,
        prediction_loss_only=True
    )

In [ ]:
if TRAIN:
    trainer.train()

In [ ]:
import torch

In [ ]:
torch.__version__

In [ ]:
import transformers
transformers.__version__

In [ ]:
#!git clone https://github.com/kwonmha/bert-vocab-builder.git
!ls

!python bert-vocab-builder/subword_builder.py \
  --corpus_filepattern "train_data.txt" \
  --output_filename git_bert_vocab.txt \
  --min_count 10

ELECTRA

In [ ]:
TRAIN = True
if TRAIN:
    import sentencepiece as spm
    spm.SentencePieceTrainer.train(input='train_data.txt', model_prefix='bert',vocab_size=20640) # Keep one space for mask token.

with open("bert.vocab", 'r', encoding='utf-8') as f:
  vc = f.read()
bert_vocab = []
for i in enumerate(vc.split("\n")):
  bert_vocab.append(i[1].split('\t')[0])
with open("bert_vocab.txt", 'w', encoding='utf-8') as f:
  f.write("\n".join(bert_vocab))


In [ ]:
from tokenizers import BertWordPieceTokenizer
tokenizer = BertWordPieceTokenizer(
    clean_text=True, handle_chinese_chars=True, strip_accents=True, lowercase=True,
)

# And then train
tokenizer.train(
    files = 'train_data.txt',
    vocab_size=15000,
    min_frequency=2,
    show_progress=True,
    special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"],
    limit_alphabet=1000,
    wordpieces_prefix="##",
)

# Save the files
tokenizer.save_model('./', 'bert_final_huggingface')

In [ ]:
from transformers import ElectraTokenizer, ElectraModel, ElectraConfig, ElectraForMaskedLM
import torch
tokenizer = ElectraTokenizer.from_pretrained("bert_final_huggingface-vocab.txt")
print(train_data[-10])
tokenizer._tokenize(train_data[-10])
#tokenizer = ElectraTokenizer.from_pretrained('google/electra-small-discriminator')
#model = ElectraModel.from_pretrained('google/electra-small-discriminator')

In [ ]:
tokenizer._tokenize(train_data[-1])

config = ElectraConfig(
    vocab_size=15000
)
model = ElectraForMaskedLM(config=config)

In [ ]:
TRAIN = True

if TRAIN:
    %%time
    from transformers import TextDataset
    dataset = TextDataset(
        tokenizer=tokenizer,
        file_path="train_data.txt",
        block_size=128,
    )

In [ ]:
if TRAIN:
    from transformers import DataCollatorForLanguageModeling

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=True, mlm_probability=0.15
    )

if TRAIN:  
    from transformers import Trainer, TrainingArguments

    training_args = TrainingArguments(
        output_dir="electra_train/",
        overwrite_output_dir=True,
        num_train_epochs=10,
        per_device_train_batch_size=64,
        save_steps=1,
        save_total_limit=2,
    )
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=dataset,
        prediction_loss_only=True,
    )

In [ ]:
if TRAIN:
    trainer.train()

In [ ]:
#!pip install tensorflow==1.11
import tensorflow as tf
print(tf.__version__)

In [ ]:
print(model)

GPT2

In [ ]:
from pathlib import Path

from tokenizers import ByteLevelBPETokenizer

paths = ["train_data.txt"]

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train(files=paths, vocab_size=4_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

# Save files to disk
tokenizer.save_model(".", "gpt2")

In [ ]:
print(tokenizer.decode(tokenizer.encode(train_data[1]).ids))
print(train_data[1])

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config
tokenizer = GPT2Tokenizer(vocab_file = 'gpt2-vocab.json', merges_file = 'gpt2-merges.txt')
print(train_data[-10])
print(tokenizer.decode(tokenizer.encode(train_data[-10])))

In [ ]:
config = GPT2Config(vocab_size=4_000)#, n_layer= 2, n_head=2
model = GPT2LMHeadModel(config=config)
print(config)
model.num_parameters()

In [ ]:
TRAIN = True

if TRAIN:
    %%time
    from transformers import TextDataset
    dataset = TextDataset(
        tokenizer=tokenizer,
        file_path="train_data.txt",
        block_size=128,
    )

In [ ]:
if TRAIN:
    from transformers import DataCollatorForLanguageModeling

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False, mlm_probability=0.15
    )

if TRAIN:  
    from transformers import Trainer, TrainingArguments

    training_args = TrainingArguments(
        output_dir="gpt2_train/",
        overwrite_output_dir=True,
        num_train_epochs=10,
        per_device_train_batch_size=2,
        save_steps=1,
        save_total_limit=2,
    )
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=dataset
    )

In [ ]:
if TRAIN:
    trainer.train()

In [ ]:
!head -5 gpt2-vocab.json

In [ ]:
!sudo rm -rf gpt2_train

In [ ]:
!nvidia-smi


Transformer XL NEXT

In [ ]:
from transformers import TransfoXLTokenizer
from transformers import TransfoXLConfig, TransfoXLLMHeadModel

# Initializing a Transformer XL configuration
config = TransfoXLConfig()
tokenizer = TransfoXLTokenizer(vocab_file='spm.vocab') ## need a simple vocab
#tokenizer.count_file("train_data.txt")
model = TransfoXLLMHeadModel(config=config)

model.num_parameters()


In [ ]:
TRAIN = True

if TRAIN:
    %%time
    from transformers import TextDataset
    dataset = TextDataset(
        tokenizer=tokenizer,
        file_path="train_data.txt",
        block_size=128,
    )

In [ ]:
if TRAIN:
    from transformers import DataCollatorForLanguageModeling

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False, mlm_probability=0.15
    )

if TRAIN:  
    from transformers import Trainer, TrainingArguments

    training_args = TrainingArguments(
        output_dir="transformer_xl_train/",
        overwrite_output_dir=True,
        num_train_epochs=10,
        per_device_train_batch_size=2,
        save_steps=1,
        save_total_limit=2,
    )
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=dataset
    )

In [ ]:
if TRAIN:
    trainer.train()

In [ ]:
|